In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

2023-04-19 17:53:51.525349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 17:53:53.011712: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 17:53:53.037489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 17:53:53.037680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [2]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [3]:
from pymongo import MongoClient
import pandas as pd

mongo = MongoClient()
data = list(mongo["data_2"]["events"].find({},{"_id": 0}))
df = pd.DataFrame(data)

ratings = df[["userid", "jobId"]]
tensor_slices = {
    "user_id": ratings.userid.values.tolist(),
    "movie_title": ratings.jobId.values.tolist()
}

movies = tf.data.Dataset.from_tensor_slices(np.unique(ratings['jobId'].to_numpy()))
ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

In [4]:
rating1st = ratings.take(30_000)
rating2nd = ratings.skip(30_000)

In [5]:
for i in movies.take(1):
    print(i)

tf.Tensor(b'Job_Fresher_Backend_Developer_DN_Fresher_3JKXHO', shape=(), dtype=string)


In [6]:
movie_titles = rating1st.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = rating1st.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [7]:
for i in movie_titles.take(1):
    print(i)

tf.Tensor(
[b'Job_Fresher_Backend_Developer_other_Fresher_YY9HOS'
 b'Job_Fresher_Backend_Developer_other_Fresher_YY9HOS'
 b'Job_Fresher_Backend_Developer_other_Fresher_YY9HOS' ...
 b'Job_Fresher_Data_Analyst_HCM_Fresher_PZ5SRX'
 b'Job_Fresher_Data_Analyst_HCM_Fresher_PZ5SRX'
 b'Job_Fresher_Data_Analyst_HCM_Fresher_PZ5SRX'], shape=(30000,), dtype=string)


In [8]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)
    print(tf.concat([user_embedding, movie_embedding], axis=1))

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [9]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [10]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [11]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
], name="user_model")

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
], name="movie_model")

metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [12]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [13]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(rating1st.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["User_Data_Other_Senior_Test9"]))
print(f"Top 3 recommendations for user User_Data_Other_Senior_Test9: {titles[0, :3]}")

Epoch 1/3
1/8 [==>...........................] - ETA: 10s - factorized_top_k/top_1_categorical_accuracy: 7.3242e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0095 - factorized_top_k/top_10_categorical_accuracy: 0.0212 - factorized_top_k/top_50_categorical_accuracy: 0.1057 - factorized_top_k/top_100_categorical_accuracy: 0.1985 - loss: 34069.6016 - regularization_loss: 0.0000e+00 - total_loss: 34069.6016

2023-04-19 17:54:28.498763: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1ff55af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-19 17:54:28.498795: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-04-19 17:54:28.501354: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-19 17:54:28.551142: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:28.580712: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster u

3/8 [==========>...................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 5.6966e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0047 - factorized_top_k/top_10_categorical_accuracy: 0.0118 - factorized_top_k/top_50_categorical_accuracy: 0.0658 - factorized_top_k/top_100_categorical_accuracy: 0.1354 - loss: 34070.4961 - regularization_loss: 0.0000e+00 - total_loss: 34070.4961

2023-04-19 17:54:28.710401: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:28.711384: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:28.800492: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

6/8 [=====================>........] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 2.8483e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0028 - factorized_top_k/top_10_categorical_accuracy: 0.0071 - factorized_top_k/top_50_categorical_accuracy: 0.0564 - factorized_top_k/top_100_categorical_accuracy: 0.1422 - loss: 34057.0111 - regularization_loss: 0.0000e+00 - total_loss: 34057.0111

2023-04-19 17:54:28.980550: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:28.997293: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:29.086879: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

8/8 [==============================] - 2s 102ms/step - factorized_top_k/top_1_categorical_accuracy: 2.3333e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0073 - factorized_top_k/top_50_categorical_accuracy: 0.0613 - factorized_top_k/top_100_categorical_accuracy: 0.1585 - loss: 28524.3572 - regularization_loss: 0.0000e+00 - total_loss: 28524.3572
Epoch 2/3
1/8 [==>...........................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0359 - factorized_top_k/top_10_categorical_accuracy: 0.0723 - factorized_top_k/top_50_categorical_accuracy: 0.2661 - factorized_top_k/top_100_categorical_accuracy: 0.4521 - loss: 32852.8945 - regularization_loss: 0.0000e+00 - total_loss: 32852.8945

2023-04-19 17:54:29.267882: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 17:54:29.338440: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


8/8 [==============================] - 1s 83ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0097 - factorized_top_k/top_5_categorical_accuracy: 0.0846 - factorized_top_k/top_10_categorical_accuracy: 0.1838 - factorized_top_k/top_50_categorical_accuracy: 0.6346 - factorized_top_k/top_100_categorical_accuracy: 0.8059 - loss: 24123.8468 - regularization_loss: 0.0000e+00 - total_loss: 24123.8468
Epoch 3/3
8/8 [==============================] - 1s 80ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0690 - factorized_top_k/top_5_categorical_accuracy: 0.3273 - factorized_top_k/top_10_categorical_accuracy: 0.5177 - factorized_top_k/top_50_categorical_accuracy: 0.9002 - factorized_top_k/top_100_categorical_accuracy: 0.9693 - loss: 20395.2949 - regularization_loss: 0.0000e+00 - total_loss: 20395.2949
Top 3 recommendations for user User_Data_Other_Senior_Test9: [b'Job_Senior_Devops_HCM_Senior_V9HAQS'
 b'Job_Fresher_Backend_Developer_other_Fresher_5BDK5X'
 b'Job_Senior_Devops_other

In [31]:
_, titles = index(np.array(["User_Fresher_Backend_DN_31"]))
print(f"Top 3 recommendations for user b'User_Fresher_Backend_DN_31: {titles[0, :20]}")

Top 3 recommendations for user b'User_Fresher_Backend_DN_31: [b'Job_Senior_Devops_HCM_Senior_V9HAQS'
 b'Job_Fresher_Backend_Developer_other_Fresher_5BDK5X'
 b'Job_Senior_Devops_other_Senior_8OMRHZ'
 b'Job_Middle_Devops_HN_Middle_Z2DAI7'
 b'Job_Fresher_Data_Analyst_other_Fresher_7QF6CC'
 b'Job_Fresher_Data_Scientist_other_Fresher_5DBKWX'
 b'Job_Fresher_Backend_Developer_DN_Fresher_3JKXHO'
 b'Job_Leader_Devops_other_Leader_SUCSJI'
 b'Job_Senior_Devops_DN_Senior_QG1ES6'
 b'Job_Fresher_Data_Analyst_HCM_Fresher_I0OQQA']


In [15]:
embedding_weights_base = model.user_model.get_layer("embedding").get_weights()[0]
vocab_base = model.user_model.get_layer("string_lookup").get_vocabulary()

In [16]:
user_ids_2nd = rating2nd.batch(1_000_000).map(lambda x: x["user_id"])
unique_user_ids_2nd = np.unique(np.concatenate(list(user_ids_2nd)))

In [24]:
new_unique_users = np.unique(list(set(unique_user_ids_2nd) - set(unique_user_ids)))
len(new_unique_users)

434

In [30]:
new_unique_users[0]

b'User_Fresher_Backend_DN_31'

In [26]:
# Generate the updated embedding matrix
updated_embedding = tf.keras.utils.warmstart_embedding_matrix(
    base_vocabulary=unique_user_ids,
    new_vocabulary=new_unique_users,
    base_embeddings=embedding_weights_base,
    new_embeddings_initializer="uniform",
)
# Update the model variable
updated_embedding_variable = tf.Variable(updated_embedding)

In [27]:
user_embedding_layer_new = tf.keras.layers.Embedding(
    len(new_unique_users), embedding_dimension
)
user_embedding_layer_new.build(input_shape=[None])
user_embedding_layer_new.embeddings.assign(updated_embedding)
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=new_unique_users, mask_token=None),
    user_embedding_layer_new
])

/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [32]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(rating2nd.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

Epoch 1/3
2/5 [===========>..................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 2.4414e-04 - factorized_top_k/top_5_categorical_accuracy: 9.7656e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0020 - factorized_top_k/top_50_categorical_accuracy: 0.0724 - factorized_top_k/top_100_categorical_accuracy: 0.1990 - loss: 39487.4570 - regularization_loss: 0.0000e+00 - total_loss: 39487.4570

2023-04-19 18:06:02.585798: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 18:06:02.669537: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 18:06:02.767929: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

5/5 [==============================] - 1s 102ms/step - factorized_top_k/top_1_categorical_accuracy: 2.2857e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0031 - factorized_top_k/top_50_categorical_accuracy: 0.0514 - factorized_top_k/top_100_categorical_accuracy: 0.1308 - loss: 29613.0371 - regularization_loss: 0.0000e+00 - total_loss: 29613.0371
Epoch 2/3


2023-04-19 18:06:02.849817: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 18:06:02.925886: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-04-19 18:06:02.995102: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

5/5 [==============================] - 0s 76ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0150 - factorized_top_k/top_5_categorical_accuracy: 0.0357 - factorized_top_k/top_10_categorical_accuracy: 0.0427 - factorized_top_k/top_50_categorical_accuracy: 0.1030 - factorized_top_k/top_100_categorical_accuracy: 0.2039 - loss: 26318.4510 - regularization_loss: 0.0000e+00 - total_loss: 26318.4510
Epoch 3/3
5/5 [==============================] - 0s 69ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0137 - factorized_top_k/top_5_categorical_accuracy: 0.0337 - factorized_top_k/top_10_categorical_accuracy: 0.0397 - factorized_top_k/top_50_categorical_accuracy: 0.1267 - factorized_top_k/top_100_categorical_accuracy: 0.2546 - loss: 25207.8841 - regularization_loss: 0.0000e+00 - total_loss: 25207.8841


In [33]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

_, titles = index(np.array(["User_Fresher_Backend_DN_31"]))
print(f"Top 3 recommendations for user b'User_Fresher_Backend_DN_31: {titles[0, :20]}")

Top 3 recommendations for user b'User_Fresher_Backend_DN_31: [b'Job_Fresher_Backend_Developer_DN_Fresher_3JKXHO'
 b'Job_Fresher_Backend_Developer_other_Fresher_5BDK5X'
 b'Job_Fresher_Backend_Engineer_other_Fresher_NM7ZJZ'
 b'Job_Fresher_Backend_Engineer_other_Fresher_QME5O3'
 b'Job_Fresher_Backend_Engineer_other_Fresher_7XN3IV'
 b'Job_Fresher_Backend_Engineer_other_Fresher_0AOC9I'
 b'Job_Fresher_Backend_Engineer_HCM_Fresher_MMKLNK'
 b'Job_Fresher_Backend_Engineer_other_Fresher_F6SGNV'
 b'Job_Fresher_Backend_Developer_other_Fresher_YY9HOS'
 b'Job_Fresher_Data_Analyst_other_Fresher_7QF6CC']
